In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [195]:
dtypes={'tourney_id': str,
        'draw_size': str,
        'winner_id': 'Int64',
        'loser_id': 'Int64',
        'winner_seed':  str,
        'winner_entry': str,
        'loser_entry': str,
        'loser_seed':  str,
       }
dtypes={**dtypes,**{'W'+str(i+1):'Int64' for i in range(5)},**{'L'+str(i+1):'Int64' for i in range(5)}}
df=pd.read_csv('csvs/atp_matches_time.csv',dtype=dtypes, date_format='%Y%m%d',parse_dates=['tourney_date'])
players=pd.read_csv('csvs/atp_players.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'csvs/atp_matches_time.csv'

In [183]:
#Filter for rank <100 and tornament:
df=df.loc[(~df.winner_rank.isna())&(~df.loser_rank.isna()),]
#df=df.loc[(df.winner_rank<101)&(df.loser_rank <101),]
df=df.loc[df.tourney_level.isin(['Grand Slam','Masters','F']),]
indices=['tourney_name', 'surface', 'draw_size','tourney_level',
       'tourney_date', 'winner_id','loser_id','l_min','w_min']
df=df.loc[:,indices]

In [184]:
while df.winner_id.value_counts().add(df.loser_id.value_counts(),fill_value=0).lt(10).any():
    v=df.winner_id.value_counts().add(df.loser_id.value_counts(),fill_value=0).ge(10)
    df=df[df.winner_id.isin(v.index[v])&df.loser_id.isin(v.index[v])]

    #filters out players with less than 50 matches played

In [185]:
df=df.rename(columns={'winner_id':'p','loser_id':'o','w_min':'p_time','l_min':'o_time',})
df.p_time=df.p_time.fillna(0)
df.o_time=df.o_time.fillna(0)
df['win']=1

In [186]:
rep=np.random.choice(df.index,int(len(df)/2),replace=False)
df.loc[rep,['p','o','p_time','o_time']]=df.loc[rep,['o','p','o_time','p_time']].values
df.loc[rep,'win']=0

In [187]:
X=pd.get_dummies(df.p,dtype=float).sub(pd.get_dummies(df.o,dtype=float),fill_value=0)
print(X.sum(axis=1).sum())
#checks the sum is 0 since tennis is a zero sum game

0.0


In [188]:
trans={players.player_id[i]:'_'.join([str(players.name_first[i]),str(players.name_last[i])]) for i in range(len(players))}
X=X.rename(columns=trans)
y=df.win
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [189]:
clf=LogisticRegression(fit_intercept=False,)
clf.fit(X_train,y_train)

LogisticRegression(fit_intercept=False)

In [170]:
def eval(y,yhat):
    return ((y-yhat)**2).mean()

In [192]:
eval(clf.predict(X_train),y_train)

np.float64(0.3229906785737566)

In [190]:
R=pd.DataFrame({'player':X.columns,'coef':clf.coef_[0]}).sort_values('coef',ascending=False)


In [193]:
R.iloc[:20]

,player,coef
665,Novak_Djokovic,3.078968
637,Rafael_Nadal,2.956948
464,Roger_Federer,2.801868
139,Pete_Sampras,2.332459
662,Andy_Murray,2.310520
1054,Carlos_Alcaraz,2.256355
504,Andy_Roddick,2.091061
721,Juan Martin_del Potro,2.006958
100,Andre_Agassi,2.002278
2,Alexander_Zverev,1.921009
